In [1]:
import pickle
import pandas as pd
import os
import torch

import betfairlightweight
from betfairlightweight import filters
from datetime import datetime
from datetime import timedelta
from dateutil import tz
import math
import numpy as np
import pandas as pd
from scipy.stats import zscore
from sklearn.linear_model import LogisticRegression
import fasttrack as ft
import importlib
import featurecreations

# Import libraries for logging in
from flumine import Flumine, clients
from flumine import BaseStrategy 
from flumine.order.trade import Trade
from flumine.order.order import LimitOrder, OrderStatus
from flumine.markets.market import Market
from betfairlightweight.resources import MarketBook
from flumine.streams.datastream import DataStream
import logging 
import re

from nltk.tokenize import regexp_tokenize

<frozen importlib._bootstrap>:228: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [2]:
seckey = "50efd775-e988-4be3-924f-87631fabdc3f"
greys = ft.Fasttrack(seckey)
today = datetime.today().strftime('%Y-%m-%d')

race_details, dog_results = greys.getBasicFormat(today, dt_end=today)
race_details = race_details[['@id', 'RaceNum', 'Distance', 'Date','RaceTime', 'RaceName', 'RaceGrade', 'Track', 'State']]
race_details['RaceId'] = race_details['@id']
dog_results = dog_results[['@id', 'RaceBox', 'DogName', 'RaceId', 'DogGrade']]
dog_results['DogId'] = dog_results['@id']
dog_results['box'] = dog_results['RaceBox']

prediction_df = pd.merge(dog_results,race_details, how='left', on='RaceId')
prediction_df.DogName = prediction_df.DogName.apply(lambda x: x.replace("'", "").replace(".", "").replace("Res", "").strip())

# x = featurecreations.generate_prediction_dataframe(r"./DATA/all-results-db.npy", prediction_df)

# x.to_pickle(f'prediction_input {today}.npy')

Valid Security Key
Getting meets for each date ..


100%|██████████| 1/1 [00:00<00:00,  2.12it/s]


Getting dog lineups ..


100%|██████████| 12/12 [00:08<00:00,  1.39it/s]


In [3]:
# Credentials to login and logging in 
trading = betfairlightweight.APIClient('nickbarlow@live.com.au','76ff98a6',app_key='JFWqJHqB4Akfi5hK')
client = clients.BetfairClient(trading, interactive_login=True)

# Login
framework = Flumine(client=client)

trading.login_interactive()

racing_filter=betfairlightweight.filters.market_filter(
        event_type_ids=["4339"], # Greyhounds
        # market_countries=["AU"], # Australia
        market_type_codes=["WIN"], # Win Markets
    )

results = trading.betting.list_events(racing_filter)

for i in results:
    print(i.event.id, i.event.name, i.market_count)

results = trading.betting.list_venues(racing_filter)
venues = [v.venue for v in results]
for i in results:
    print(i, i.venue,i.market_count)

results_list = []

for v in venues:
    racing_filter=betfairlightweight.filters.market_filter(
        event_type_ids=["4339"], # Greyhounds
        market_countries=["AU"], # Australia
        market_type_codes=["WIN"], # Win Markets
        venues=[v]
    )

    results = trading.betting.list_market_catalogue(
            market_projection=[
                "RUNNER_DESCRIPTION", 
                "RUNNER_METADATA", 
                "COMPETITION", 
                "EVENT", 
                "EVENT_TYPE", 
                "MARKET_DESCRIPTION", 
                "MARKET_START_TIME",
            ],
            filter=racing_filter,
            max_results=110,
        )

    results_list.extend(results)

df_list = []

box_change_list = []
pattern1 = r'(?<=<br>Dog ).+?(?= starts)'
pattern2 = r"(?<=\bbox no. )(\w+)"

for i in results_list:
    print(f"{i.market_id,i.market_name,i.market_start_time.hour, i.market_start_time.minute, i.event.venue,  i.description.market_type} ")
    market_id = i.market_id
    race_num = int(re.sub("[^0-9]", "", i.market_name.split(' ',1)[0] ))
    track =  i.event.venue
    dist = i.market_name.split(' ',2)[0]
    print(race_num)
    if i.description.clarifications:
        print(i.description.clarifications.replace("<br> Dog","<br>Dog"))
        box_change_list.append((track, race_num,i.description.clarifications.replace("<br> Dog","<br>Dog")))

    for dog in i.runners:
        print(f"id = {dog.selection_id}, name = {dog.runner_name.split(' ',1)[1].upper()}")
        df_list.append([market_id, track, dist, race_num, dog.selection_id,dog.runner_name.split('.',1)[0].upper(), dog.runner_name.split(' ',1)[1].upper()])
df = pd.DataFrame(data = df_list, columns=['market_id', 'track', 'dist', 'race_num', 'runner_id','box_bf', 'runnner_name'])

32253485 Richmond (AUS) 9th Apr 8
32253807 Pelaw Grange 9th Apr 10
32253481 Grafton (AUS) 9th Apr 10
32253834 Harlow 9th Apr 12
32253355 Broken Hill (AUS) 9th Apr 11
32248771 Mullingar 9th Apr 10
32253948 Harlow 9th Apr 12
32253916 Henlow 9th Apr 11
32253432 Darwin (AUS) 9th Apr 8
32253817 Doncaster 9th Apr 12
32253850 Sunderland 9th Apr 12
32253339 Gawler (AUS) 9th Apr 9
32253942 Kinsley 9th Apr 12
32253431 Cannington (AUS) 9th Apr 12
32253872 Towcester 9th Apr 14
32253425 Sandown Park (AUS) 9th Apr 6
32253905 Central Park 9th Apr 11
VenueResult Gawler 9
VenueResult Grafton 10
VenueResult Broken Hill 11
VenueResult Cannington 12
VenueResult Sandown Park 6
VenueResult Richmond 8
VenueResult Doncaster 12
VenueResult Darwin 8
VenueResult Harlow 24
VenueResult Sunderland 12
VenueResult Pelaw Grange 10
VenueResult Towcester 14
VenueResult Central Park 11
VenueResult Mullingar 10
VenueResult Henlow 11
VenueResult Kinsley 12
('1.212568376', 'R4 400m Gr6', 8, 34, 'Gawler', 'WIN') 
4
id = 5385

In [4]:
tracks_FT = prediction_df.Track.unique()

In [5]:
races_ft_df = df[['track','race_num', 'box_bf']].copy()
races_ft_df.columns = ['Track', 'RaceNum', 'RaceBox']
races_ft_df.replace('Wagga','Wagga Wagga',inplace = True)

if 'Richmond Straight' in tracks_FT:
    races_ft_df.replace('Richmond','Richmond Straight',inplace = True)
    print("replacing richmond")

races_ft_df

,Track,RaceNum,RaceBox
0,Gawler,4,1
1,Gawler,4,2
2,Gawler,4,3
3,Gawler,4,4
4,Gawler,4,5
...,...,...,...
475,Darwin,8,1
476,Darwin,8,2
477,Darwin,8,4
478,Darwin,8,6


In [6]:
races_ft_df.RaceBox = pd.to_numeric(races_ft_df.RaceBox)

In [7]:
races_ft_df.dtypes

Track      object
RaceNum     int64
RaceBox     int64
dtype: object

In [8]:
box_change_df = pd.DataFrame.from_records(box_change_list, columns = ['Track', 'RaceNum', 'string'])
#box_change_df.drop(index=0, inplace=True)
box_change_df = box_change_df[~(box_change_df['string']==' ')]
box_change_df['runner_name'] = box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern1)[0])
box_change_df['runner_number'] = box_change_df['runner_name'].apply(lambda x: x[:(x.find(" ") - 1)].upper())
box_change_df['runner_name'] = box_change_df['runner_name'].apply(lambda x: x[(x.find(" ") + 1):].upper())
box_change_df['RaceBox'] = pd.to_numeric(box_change_df['string'].apply(lambda x: regexp_tokenize(x, pattern2)[0]))
prediction_df.RaceNum = pd.to_numeric(prediction_df.RaceNum)
prediction_df.RaceBox = pd.to_numeric(prediction_df.RaceBox)
prediction_df = prediction_df.reset_index(drop=True).set_index(['Track', 'RaceNum', 'RaceBox'])
prediction_df = prediction_df.drop(index=list(box_change_df[['Track','RaceNum','RaceBox']].itertuples(index=False)), errors='ignore').reset_index()
prediction_df = prediction_df.merge(races_ft_df, on=['Track', 'RaceNum', 'RaceBox'])
merged = prediction_df.merge(how='left', right=box_change_df, right_on=['runner_name', 'RaceNum'], left_on=['DogName', 'RaceNum'])
merged['Box'] = merged.apply(lambda x: int(x['box']) if np.isnan(x['RaceBox_y']) else int(x['RaceBox_y']), axis =1)
merged['race_time'] = pd.to_datetime(merged['RaceTime'])
new_form = merged[merged['Box']<=8][['Track_x','State','DogId','DogName','DogGrade', 'RaceId','Distance', 'Date', 'Box','race_time','RaceNum']].rename(columns={'Track_x':'Track', 'RaceNum':'race_num'})

In [9]:
new_form

,Track,State,DogId,DogName,DogGrade,RaceId,Distance,Date,Box,race_time,race_num
0,Sandown Park,VIC,536729320,SUMMER DAZE,5,888991939,595m,09 Apr 23,1,2023-04-09 18:49:00,5
1,Sandown Park,VIC,527460319,SURPRISING,4,888991939,595m,09 Apr 23,2,2023-04-09 18:49:00,5
2,Sandown Park,VIC,665757358,PIRATE KINGSLEY,5,888991939,595m,09 Apr 23,4,2023-04-09 18:49:00,5
3,Sandown Park,VIC,568419583,ALBIES CRUISER,5,888991939,595m,09 Apr 23,5,2023-04-09 18:49:00,5
4,Sandown Park,VIC,655391168,DOTTYS GIFT,5,888991939,595m,09 Apr 23,7,2023-04-09 18:49:00,5
...,...,...,...,...,...,...,...,...,...,...,...
395,Darwin,NT,559092462,BARNEY SUN,2,889002506,383m,09 Apr 23,1,2023-04-09 21:33:00,8
396,Darwin,NT,623993009,NICK THE QUICK,1,889002506,383m,09 Apr 23,2,2023-04-09 21:33:00,8
397,Darwin,NT,511163286,BOP BOP,3,889002506,383m,09 Apr 23,4,2023-04-09 21:33:00,8
398,Darwin,NT,613675909,GUDGEON PIN,2,889002506,383m,09 Apr 23,6,2023-04-09 21:33:00,8


In [10]:
box_change_df

,Track,RaceNum,string,runner_name,runner_number,RaceBox
0,Gawler,5,<additionalinfo><br>Box changes:<br>Dog 9. Tre...,TRENDING STRIKER,9,3
1,Gawler,7,<additionalinfo><br>Box changes:<br>Dog 9. Res...,RESURGENCE,9,5
2,Gawler,9,<additionalinfo><br>Box changes:<br>Dog 10. We...,WESTAR AILEEN,10,1
3,Cannington,5,<additionalinfo><br>Box changes:<br>Dog 10. Re...,RETHINK IT,10,8
4,Cannington,9,<additionalinfo><br>Box changes:<br>Dog 9. Hel...,HELLO IM AERO,9,1
5,Richmond,7,<additionalinfo><br>Box changes:<br>Dog 9. Bla...,BLACK ROCKET,9,3
6,Richmond,8,<additionalinfo><br>Box changes:<br>Dog 9. Luc...,LUCAYAN,9,1
7,Richmond,9,<additionalinfo><br>Box changes:<br>Dog 9. Jud...,JUDGMENT FAY,9,3
8,Richmond,12,<additionalinfo><br>Box changes:<br>Dog 10. Lo...,LOLA PEPPER,10,7


In [11]:
importlib.reload(featurecreations)
test_out = featurecreations.generate_prediction_dataframe_v2('results-df-merged-prices.npy', new_form)

fin 30
fin 5
fin 1
fin 10_dist
fin 10 box
fin 10 track_box
fin 10 track_dist
['speed_avg_30', 'speed_max_30', 'split_speed_avg_30', 'split_speed_max_30', 'split_margin_avg_30', 'margin_avg_30', 'first_out_avg_30', 'post_change_avg_30', 'races_30', 'wins_30', 'speed_avg_5', 'speed_max_5', 'split_speed_avg_5', 'split_speed_max_5', 'split_margin_avg_5', 'margin_avg_5', 'first_out_avg_5', 'post_change_avg_5', 'races_5', 'wins_5', 'speed_avg_1', 'speed_max_1', 'split_speed_avg_1', 'split_speed_max_1', 'split_margin_avg_1', 'margin_avg_1', 'first_out_avg_1', 'post_change_avg_1', 'races_1', 'wins_1', 'speed_avgdist_10', 'speed_maxdist_10', 'split_speed_avgdist_10', 'split_speed_maxdist_10', 'split_margin_avgdist_10', 'margin_avgdist_10', 'first_out_avgdist_10', 'post_change_avgdist_10', 'racesdist_10', 'winsdist_10', 'speed_avgbox_10', 'speed_maxbox_10', 'split_speed_avgbox_10', 'split_speed_maxbox_10', 'split_margin_avgbox_10', 'margin_avgbox_10', 'first_out_avgbox_10', 'post_change_avgbox_1

In [12]:
new_form["date"] =  pd.to_datetime(prediction_df["Date"], format="%d %b %y").dt.date
new_form["date"] 

0      2023-04-09
1      2023-04-09
2      2023-04-09
3      2023-04-09
4      2023-04-09
          ...    
395    2023-04-09
396    2023-04-09
397    2023-04-09
398    2023-04-09
399    2023-04-09
Name: date, Length: 399, dtype: object

In [13]:
test_out = test_out[test_out['raceid'].isin( new_form['RaceId'])]
test_out.to_pickle(f'testing new outs {today}.npy')

In [14]:
test_out100 = test_out.head()

In [15]:
test_out

,dogid,dog_name,raceid,race_num,race_grade,date,race_time,trackOHE,track_name,state,...,dist,bfSP,runtime,prev_race,prev_race_date,prev_race_track,place,margin,box,stats
15776,536729320,SUMMER DAZE,888991939,5.0,NaN,2023-04-09,2023-04-09 18:49:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Sandown Park,VIC,...,595.0,NaN,NaN,886813594,2023-04-02 00:00:00,Sandown Park,NaN,NaN,1,"[1.0, 25.5, 5.0, 17.37692205097138, 17.7127454..."
15777,527460319,SURPRISING,888991939,5.0,NaN,2023-04-09,2023-04-09 18:49:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Sandown Park,VIC,...,595.0,NaN,NaN,885838643,2023-03-29 00:00:00,The Meadows,NaN,NaN,2,"[2.0, 29.5, 4.0, 17.284340669216128, 17.736786..."
15778,665757358,PIRATE KINGSLEY,888991939,5.0,NaN,2023-04-09,2023-04-09 18:49:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Sandown Park,VIC,...,595.0,NaN,NaN,884739521,2023-03-30 00:00:00,Sandown Park,NaN,NaN,4,"[4.0, 31.8, 5.0, 17.16714355621553, 17.4534574..."
15779,568419583,ALBIES CRUISER,888991939,5.0,NaN,2023-04-09,2023-04-09 18:49:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Sandown Park,VIC,...,595.0,NaN,NaN,887147260,2023-04-04 00:00:00,Warragul,NaN,NaN,5,"[5.0, 26.1, 5.0, 17.28482176008594, 17.5840978..."
15780,655391168,DOTTYS GIFT,888991939,5.0,NaN,2023-04-09,2023-04-09 18:49:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Sandown Park,VIC,...,595.0,NaN,NaN,881902294,2023-03-16 00:00:00,Sandown Park,NaN,NaN,7,"[7.0, 27.9, 5.0, 17.16432193472789, 17.6583493..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16170,559092462,BARNEY SUN,889002506,8.0,NaN,2023-04-09,2023-04-09 21:33:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Darwin,NT,...,383.0,NaN,NaN,886871248,2023-04-05 00:00:00,Darwin,NaN,NaN,1,"[1.0, 30.6, 2.0, 17.03806447823313, 17.5182481..."
16171,623993009,NICK THE QUICK,889002506,8.0,NaN,2023-04-09,2023-04-09 21:33:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Darwin,NT,...,383.0,NaN,NaN,882009451,2023-04-02 00:00:00,Darwin,NaN,NaN,2,"[2.0, 32.4, 1.0, 17.227298353861553, 17.572304..."
16172,511163286,BOP BOP,889002506,8.0,NaN,2023-04-09,2023-04-09 21:33:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Darwin,NT,...,383.0,NaN,NaN,792134968,2022-06-18 00:00:00,The Meadows,NaN,NaN,4,"[4.0, 33.4, 3.0, 17.445398139820707, 18.087855..."
16173,613675909,GUDGEON PIN,889002506,8.0,NaN,2023-04-09,2023-04-09 21:33:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Darwin,NT,...,383.0,NaN,NaN,886871248,2023-04-05 00:00:00,Darwin,NaN,NaN,6,"[6.0, 26.3, 2.0, 17.087355555613758, 17.381615..."


In [16]:
tail = test_out.tail(2000)

In [17]:
test_out_pred  = test_out[test_out['raceid'].isin( new_form['RaceId'])]

In [18]:
stats_col = ['speed_avg_30', 'speed_max_30', 'split_speed_avg_30', 'split_speed_max_30', 'split_margin_avg_30', 'margin_avg_30', 'races_30', 'wins_30', 'speed_avg_5', 'speed_max_5', 'split_speed_avg_5', 'split_speed_max_5', 'split_margin_avg_5', 'margin_avg_5', 'races_5', 'wins_5', 'speed_avg_1', 'speed_max_1', 'split_speed_avg_1', 'split_speed_max_1', 'split_margin_avg_1', 'margin_avg_1', 'races_1', 'wins_1', 'speed_avgdist_10', 'speed_maxdist_10', 'split_speed_avgdist_10', 'split_speed_maxdist_10', 'split_margin_avgdist_10', 'margin_avgdist_10', 'racesdist_10', 'winsdist_10', 'speed_avgbox_10', 'speed_maxbox_10', 'split_speed_avgbox_10', 'split_speed_maxbox_10', 'split_margin_avgbox_10', 'margin_avgbox_10', 'racesbox_10', 'winsbox_10']

In [19]:
len(stats_col)

40

In [20]:
test_out_pred[test_out_pred['state']=="NSW"]

,dogid,dog_name,raceid,race_num,race_grade,date,race_time,trackOHE,track_name,state,...,dist,bfSP,runtime,prev_race,prev_race_date,prev_race_track,place,margin,box,stats
15815,650078113,SACRED SILENCE,886871396,5.0,NaN,2023-04-09,2023-04-09 18:52:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Richmond,NSW,...,520.0,NaN,NaN,885078752,2023-04-02 00:00:00,Richmond,NaN,NaN,1,"[1.0, 27.4, 5.0, 17.1138897932725, 17.72428884..."
15816,451768307,EMERALD GREEN,886871396,5.0,NaN,2023-04-09,2023-04-09 18:52:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Richmond,NSW,...,520.0,NaN,NaN,880939742,2023-03-25 00:00:00,Wentworth Park,NaN,NaN,2,"[2.0, 32.2, 5.0, 17.177153152477672, 17.885562..."
15817,675307061,OLD EASY,886871396,5.0,NaN,2023-04-09,2023-04-09 18:52:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Richmond,NSW,...,520.0,NaN,NaN,885078752,2023-04-02 00:00:00,Richmond,NaN,NaN,3,"[3.0, 32.8, 5.0, 17.256819121649176, 18.254364..."
15818,540582166,ZIPPING RUBBLE,886871396,5.0,NaN,2023-04-09,2023-04-09 18:52:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Richmond,NSW,...,520.0,NaN,NaN,826268241,2022-09-16 00:00:00,Traralgon,NaN,NaN,4,"[4.0, 34.6, 4.0, 17.51192037557857, 18.4090909..."
15819,602825248,INSIDE DART,886871396,5.0,NaN,2023-04-09,2023-04-09 18:52:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Richmond,NSW,...,520.0,NaN,NaN,882009045,2023-03-18 00:00:00,Wentworth Park,NaN,NaN,5,"[5.0, 32.1, 5.0, 17.221132049061396, 17.761989..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15950,638803578,ROBELL MAGIC,886871479,12.0,NaN,2023-04-09,2023-04-09 21:44:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Grafton,NSW,...,350.0,NaN,NaN,884739547,2023-03-30 00:00:00,Casino,NaN,NaN,4,"[4.0, 26.6, 5.0, 17.376798049025787, 17.709563..."
15951,587948149,LA FOLIE,886871479,12.0,NaN,2023-04-09,2023-04-09 21:44:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Grafton,NSW,...,350.0,NaN,NaN,884739554,2023-03-30 00:00:00,Casino,NaN,NaN,5,"[5.0, 26.3, 5.0, 16.874971732444116, 17.709563..."
15952,558491833,GYPSY ROAD,886871479,12.0,NaN,2023-04-09,2023-04-09 21:44:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Grafton,NSW,...,350.0,NaN,NaN,884739754,2023-04-02 00:00:00,Grafton,NaN,NaN,6,"[6.0, 27.1, 5.0, 16.9978742621907, 17.61597181..."
15953,680210716,SHES ALL CHARM,886871479,12.0,NaN,2023-04-09,2023-04-09 21:44:00,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",Grafton,NSW,...,350.0,NaN,NaN,885078802,2023-04-04 00:00:00,Casino,NaN,NaN,7,"[7.0, 23.9, 5.0, 17.255583123593382, 17.678255..."


In [21]:
test_out_pred.to_pickle(f'testing new outs {today}.npy')

In [22]:
len(test_out_pred.trackOHE.iloc[0])

69

In [23]:
test_out_pred[test_out_pred['dog_name']=="John"]

,dogid,dog_name,raceid,race_num,race_grade,date,race_time,trackOHE,track_name,state,...,dist,bfSP,runtime,prev_race,prev_race_date,prev_race_track,place,margin,box,stats


In [24]:
rich = test_out_pred[test_out_pred['track_name']=='The Gardens']

In [25]:
len(test_out_pred.stats.iloc[0])

77

In [26]:
len(test_out_pred.trackOHE.iloc[0])

69